In [1]:
%load_ext autoreload
%autoreload 2

In [19]:
from __future__ import annotations

import jax.numpy as jnp
from pathlib import Path
from diffwake.diffwake_jax.model import load_input, create_state, turbine_powers, alter_yaw_angles
from diffwake.diffwake_jax.simulator import simulate

# 1. Define paths to configuration files
data_dir = Path("./data/horn")
farm_cfg_path = data_dir / "cc_hornsRev.yaml"
turbine_cfg_path = data_dir / "vestas_v802MW.yaml"

# 2. Load the base configuration
cfg = load_input(farm_cfg_path, turbine_cfg_path)

# 3. Define a simple 3-turbine layout (aligned along the X-axis)
# Placing them 5D apart: 0m, 400m, 800m.
layout_x = jnp.array([0.0, 400.0, 800.0])
layout_y = jnp.array([0.0, 0.0, 0.0])

# 4. Define flow conditions
wind_speed = jnp.array([8.0])
wind_dir = jnp.array([jnp.deg2rad(270.0)]) # Wind from West
ti = jnp.array([0.06])

# 5. Update the configuration
cfg = cfg.set(
    layout_x=layout_x,
    layout_y=layout_y,
    wind_speeds=wind_speed,
    wind_directions=wind_dir,
    turbulence_intensities=ti
)

# 6. Create the simulation state
state = create_state(cfg)

# --- Baseline: Unyawed Farm ---
yaw_angles_unyawed = jnp.zeros((1, 3)) # (n_findex, n_turbines)
state_unyawed = alter_yaw_angles(yaw_angles_unyawed, state)

result_unyawed = simulate(state_unyawed)
state_final_unyawed = state_unyawed.replace(flow=state_unyawed.flow.replace(u=result_unyawed.u_sorted))
powers_unyawed = turbine_powers(state_final_unyawed)[0] # First (and only) wind condition

# --- Case: Yawed Farm ---
# Specify yaw angles (in degrees) for each turbine
yaw_angles_deg = jnp.array([[20.0, 10.0, 0.0]]) # Try steering wakes away from downstream turbines
yaw_angles_rad = jnp.deg2rad(yaw_angles_deg)

state_yawed = alter_yaw_angles(yaw_angles_rad, state)

result_yawed = simulate(state_yawed)
state_final_yawed = state_yawed.replace(flow=state_yawed.flow.replace(u=result_yawed.u_sorted))
powers_yawed = turbine_powers(state_final_yawed)[0]

# 7. Compare and display results
print("Simulation Comparison (3 turbines in a row):")
print(f"Layout X: {layout_x} m")
print(f"Wind Speed: {wind_speed[0]} m/s, Direction: 270 deg")
print("-" * 50)
print(f"{'Turbine':<10} | {'Unyawed (kW)':<15} | {'Yawed (kW)':<15} | {'Yaw (deg)':<10}")
print("-" * 50)
for i in range(len(layout_x)):
    print(f"{i:<10} | {powers_unyawed[i]/1e3:15.2f} | {powers_yawed[i]/1e3:15.2f} | {yaw_angles_deg[0, i]:10.1f}")

total_unyawed = jnp.sum(powers_unyawed) / 1e3
total_yawed = jnp.sum(powers_yawed) / 1e3
diff = total_yawed - total_unyawed
perc_diff = (diff / total_unyawed) * 100

print("-" * 50)
print(f"{'Total':<10} | {total_unyawed:15.2f} | {total_yawed:15.2f} |")
print("-" * 50)
print(f"Difference: {diff:.2f} kW ({perc_diff:+.2f}%)")


TypeError: scan body function carry input and carry output must have the same pytree structure, but they differ:

The input carry component loop_carry[1] is a <class 'diffwake.diffwake_jax.util.CCDynamicState'> but the corresponding component of the carry output is a <class 'diffwake.diffwake_jax.util.CCDynamicState'>, so their Python types differ.

Revise the function so that the carry output has the same pytree structure as the carry input.

In [20]:
c

SyntaxError: invalid syntax (628468241.py, line 1)